In [2]:
import pandas as pd
from dateutil import parser

The basic idea in our trading strategy is open the position when the market is extreme greed or fear, the volatility and the basis is under an acceptable level 

Distribution of Fear Index

In [3]:
btc_usdt = pd.read_csv("./edav_onchaindata/BTC-USDT.csv")
btc_usdt["time"] = pd.to_datetime(btc_usdt["time"])
btc_usdt = btc_usdt.sort_values(by = "time")
# btc_usdt["price"] = (btc_usdt["high"].to_numpy() + btc_usdt["low"].to_numpy() + btc_usdt["close"].to_numpy())/3
btc_usdt["price"] =  btc_usdt["close"].to_numpy()
btc_usdt = btc_usdt[["time", "price"]]

fear_df = pd.read_csv("./edav_onchaindata/fundamentalDf.csv")
fear_df["time"] = pd.to_datetime(fear_df["time"]).dt.date
fear_df = fear_df[["fear", "time"]]

In [7]:
a = ['a', 'b', 'c', 'd']
b = [2,3,4,1]


In [10]:
sorted(zip(a,b), key = lambda x :x[1], reverse= True)[0]

('c', 4)

In [160]:
from os import listdir
from os.path import isfile, join
futures = [f for f in listdir("./edav_onchaindata/future-swap") if isfile(join("./edav_onchaindata/future-swap", f))]

In [217]:
future_dfs = {}
for file_name in futures:
    delvery_time = "20" + file_name[9:11] + "-" + file_name[11:13] + "-" + file_name[13:15]
    delvery_time = parser.parse(delvery_time).date()
    delvery_time = delvery_time
    df = pd.read_csv("./edav_onchaindata/future-swap/{}".format(file_name))
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values(by = "time")
#     df["future_price"] = (df["high"].to_numpy() + df["low"].to_numpy() + df["close"].to_numpy())/3
    df["future_price"] = df["close"].to_numpy()
    df = df[["time", "future_price"]]
    df = pd.merge(df, btc_usdt, how = "inner", left_on="time", right_on="time")
    df["basis_rate"] = (df["future_price"] - df["price"])/df["price"] * 100
    df["time"] = df["time"].dt.date
    df = df.groupby("time").tail(1)
    default_funding_rate = np.array((delvery_time - df.time) / np.timedelta64(1, 'D')) * 0.03
    df["recal_basis_rate"] = df["basis_rate"].to_numpy() - default_funding_rate
    df = df.iloc[2:len(df)-3]
    df = pd.merge(df, fear_df, how = "inner", left_on="time", right_on="time")
    future_dfs[delvery_time] = df

In [218]:
deliveries = list(future_dfs.keys())

In [227]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=8, cols=1, 
                    specs=[[{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}],
                            [{"secondary_y": True}], [{"secondary_y": True}]])

for i in range(4):
    for j in range(2):
        df = future_dfs[deliveries[2*i+j]]
        fig.add_trace(
            go.Scatter(x=df.time, y=df.recal_basis_rate, name="recal_basis_rate"),
            row=2*i+j+1, col=1,
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(x=df.time, y=df.fear, name = "fear"),
            row=2*i+j+1, col=1,
            secondary_y=True,
        )
fig.update_layout(height=2400, width=900)
fig.show()

In [230]:
df = future_dfs[deliveries[0]]
df["fear "]

,time,future_price,price,basis_rate,recal_basis_rate,fear
0,2020-03-02,9234.2,8911.1,3.625815,0.145815,38
1,2020-03-03,9060.0,8759.2,3.434104,-0.015896,38
2,2020-03-04,9018.9,8750.8,3.063720,-0.356280,40
3,2020-03-05,9388.7,9054.5,3.690982,0.300982,41
4,2020-03-06,9497.4,9133.1,3.988788,0.628788,39
...,...,...,...,...,...,...
109,2020-06-19,9311.2,9312.0,-0.008591,-0.218591,39
110,2020-06-20,9365.2,9358.4,0.072662,-0.107338,38
111,2020-06-21,9296.5,9295.7,0.008606,-0.141394,37
112,2020-06-22,9690.2,9684.0,0.064023,-0.055977,38


In [60]:
np.array((datetime.datetime(2020, 6, 26, 0, 0) - B.index) / np.timedelta64(1, 'D')) * 0.3

array([35.4, 35.1, 34.8, 34.5, 34.2, 33.9, 33.6, 33.3, 33. , 32.7, 32.4,
       32.1, 31.8, 31.5, 31.2, 30.9, 30.6, 30.3, 30. , 29.7, 29.4, 29.1,
       28.8, 28.5, 28.2, 27.9, 27.6, 27.3, 27. , 26.7, 26.4, 26.1, 25.8,
       25.5, 25.2, 24.9, 24.6, 24.3, 24. , 23.7, 23.4, 23.1, 22.8, 22.5,
       22.2, 21.9, 21.6, 21.3, 21. , 20.7, 20.4, 20.1, 19.8, 19.5, 19.2,
       18.9, 18.6, 18.3, 18. , 17.7, 17.4, 17.1, 16.8, 16.5, 16.2, 15.9,
       15.6, 15.3, 15. , 14.7, 14.4, 14.1, 13.8, 13.5, 13.2, 12.9, 12.6,
       12.3, 12. , 11.7, 11.4, 11.1, 10.8, 10.5, 10.2,  9.9,  9.6,  9.3,
        9. ,  8.7,  8.4,  8.1,  7.8,  7.5,  7.2,  6.9,  6.6,  6.3,  6. ,
        5.7,  5.4,  5.1,  4.8,  4.5,  4.2,  3.9,  3.6,  3.3,  3. ,  2.7,
        2.4,  2.1,  1.8,  1.5,  1.2,  0.9,  0.6,  0.3,  0. ])

In [ ]:
def fuck(X, shares):
    X_dic = {}
    for x in X:
        